In [1]:
import importlib
import pandas as pd
import sys
import numpy as np
import pythoncom
from win32com.client import VARIANT
import pickle
import solidworks_functions as sf
import os
from tqdm import tqdm
from IPython.display import clear_output
importlib.reload(sf)

sys.path.append('../Code/Gradient NN/src')
import geometric_functions as gf

## Test

In [2]:
# Set new values in the parameter file
params = {
    "radius": 10
}
params_box = {"width": 100, 
              "height": 100, 
              "length": 100}
output_path = "./STL/test.stl"

model = sf.Create_geometry("ball", output_path, params)



✅ Trying to open part: c:\Users\magnu\OneDrive\Dokumenter\GitHub\R-D\Drawings\Ball\Ball.SLDPRT
💾 Saving to: c:\Users\magnu\OneDrive\Dokumenter\GitHub\R-D\Drawings\STL\test.stl
Geometry created and saved to ./STL/test.stl


In [3]:
area = sf.get_surface_area(model, "mm2")

Surface Area: 1256.637061435916 mm^2


In [3]:
import meshlab_functions as mf
importlib.reload(mf)

<module 'meshlab_functions' from 'c:\\Users\\magnu\\OneDrive\\Dokumenter\\GitHub\\R-D\\Drawings\\meshlab_functions.py'>

In [4]:
output_path_xyz = "./STL/test.xyz"
mf.sample_stl_by_point_distance(output_path, output_path_xyz, 1)


NameError: name 'output_path' is not defined

In [5]:
from scipy.spatial import KDTree

In [6]:
output_path_xyz = "./STL/test.xyz"
pointcloud = np.loadtxt(output_path_xyz)
mf.save_neighborhood_to_txt(pointcloud[:,:3])

Saved neighborhood to neighborhood.txt


In [36]:
kdtree2 = KDTree(pointcloud)
neighborhood, _, _  = gf.find_neighbors(kdtree2, 10, pointcloud,20000)

mf.save_neighborhood_to_txt(neighborhood, "test.txt")

Saved neighborhood to test.txt


In [37]:
# Calculate curvature
features_jax = gf.compute_geometric_properties(neighborhood)
features = np.array(features_jax)  # Convert to regular NumPy array
features = features.tolist()  
features.append(2)

print(features)

[0.045583732426166534, 0.904679536819458, 0.004213784355670214, 0.900465726852417, 0.0953204408288002, 0.9129748344421387, 2]


In [7]:
gf.calculate_point_density(pointcloud=pointcloud, surface_area=area)

4.103014432909066

In [13]:
pc_with_holes = mf.create_mesh_holes(pointcloud, 10, 20)
mf.save_neighborhood_to_txt(pc_with_holes, "PC_holes.txt")

Saved neighborhood to PC_holes.txt


In [ ]:
Noise_PC = mf.create_noise(pc_with_holes[:,:3], 0.2, 0.1)

mf.save_neighborhood_to_txt(Noise_PC, "noisePC.txt")

Saved neighborhood to noisePC.txt


In [1]:
def find_neighbors(kdtree, index, points, searchK):
    _, indices = kdtree.query([points[index]], k=searchK)
    neighborhood = points[indices[0]]
    aligned_neighborhood = standardize_patch(neighborhood)
    
    return aligned_neighborhood, neighborhood, indices[0]

In [ ]:
output_path = "./STL/test.stl"
output_path_xyz = "./STL/test.xyz"



for i in tqdm(range(10, 50, 1), desc="Generating geometries"):
    params["radius"] = i
    sf.Create_geometry("ball", output_path, params)
    mf.sample_stl_by_point_distance(output_path, output_path_xyz, 1)
    point_cloud = pd.read_csv(output_path_xyz)
    clear_output(wait=True)
    


Generating geometries: 100%|██████████| 2/2 [00:19<00:00,  9.70s/it]
